In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
import wikipediaapi as wk
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

from prompts import load_template
from utils import create_experiment_file_name

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WIKI_USER_AGENT = os.getenv("WIKI_USER_AGENT")

DATA_DIR = "data"
RESULT_DIR = "experiments"
TARGET_DATA = "train_v2.0.1.csv"
EXP_VERSION = "v1.0.1"  # 컨벤션에 따라 새롭게 실험할 경우 버저닝 정보를 기록해주세요.

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"

In [ ]:
datasets = pd.read_csv(os.path.join(DATA_DIR, TARGET_DATA))
datasets.head()

,id,paragraph,question,choices,answer,question_plus
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"['ㄱ, ㄴ', 'ㄱ, ㄷ', 'ㄴ, ㄹ', 'ㄷ, ㄹ']",2,ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 ...
1,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"['이곳에 대장도감을 설치하여 재조대장경을 만들었다.', '지눌이 이곳에서 수선사 ...",4,NaN
2,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"['살수에서 수의 군대를 물리쳤다 .', '김춘추 의 신라 왕위 계승을 지원하였다 ...",2,NaN
3,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"['사창제를 실시하였다 .', '대전회통을 편찬하였다 .', '비변사의 기능을 강화...",3,NaN
4,generation-for-nlp-431,(가)의 사신 고제덕 등이 일본에 와서 왕이 보낸 국서를 전하였다. 그 국서에 이르...,(가) 국가에 대한 설명으로 옳은 것은?,"['나 당연합군의 공격으로 멸망하였다 .', '9주 5소경의 지방 행정 구역을 두었...",3,NaN


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="wikipedia_search_keyword.txt", template_type="keyword_extraction")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, max_tokens=None, timeout=None, max_retries=2, api_key=OPENAI_API_KEY
)

keyword_extractor = prompt | llm

In [ ]:
def extract_keyword(data: pd.Series) -> list[str]:
    """
    문제 해결에 필요한 키워드 5개를 추출하고 결과를 배열로 반환하는 함수

    Args:
        data (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나

    Returns:
        list[str]: 추출된 5개 키워드
    """
    response = keyword_extractor.invoke(
        {
            "paragraph": data[PARAGRAPH],
            "question": data[QUESTION],
            "question_plus": data[QUESTION_PLUS],
            "choices": data[CHOICES],
        },
    )
    response_split = response.content.split(",")
    keywords = [keyword.strip() for keyword in response_split]
    return keywords


sampled_datasets: pd.DataFrame = datasets.sample(20, random_state=1004)
sampled_datasets["keywords"] = sampled_datasets.apply(extract_keyword, axis=1)
sampled_datasets.head()

,id,paragraph,question,choices,answer,question_plus,keywords
1190,generation-for-nlp-2237,대안학교는 중·고교만 있는 게 아니다. 20대를 위한 대안학교도 있다. 급변하는 지...,융합인재사관학교(융합사)의 설립 목적은 무엇인가?,"['청년인재 양성', '대학 진학 지원', '취업 알선', '예술 교육 강화', '...",1,NaN,"[융합인재사관학교, 청년인재, 교육, 멀티플레이어, 서바이벌 스토리]"
242,generation-for-nlp-762,대일 수출 달러에 대한 수요 달러의 가치,일본 경제가 장기적인 경기 침체를 겪고 있다고 가정하겠습니다. 일본 가계 소득 감소...,"['감소 \xa0\xa0\xa0 감소 \xa0\xa0\xa0 감소', '감소 \xa...",1,NaN,"[대일 수출, 달러, 엔화, 경기 침체, 가계 소득]"
876,generation-for-nlp-1712,추가 서비스마다 요금을 받기로 유명한 저비용항공사(LCC) 스피릿항공이 미국 소비자...,미국 소비자 만족도 조사에서 최하위를 기록한 저비용항공사는 무엇인가?,"['스피릿항공', '제트블루항공', '델타항공', '아메리칸항공', '유나이티드항공']",1,NaN,"[스피릿항공, 제트블루항공, 소비자만족도, 저비용항공사, ACSI]"
264,generation-for-nlp-791,희소성은 다음과 같이 정의할 수 있습니다.,희소성에 대한 정의로 옳은 것은?,"['제한된 욕구와 제한된 경제적 자원 간 차이.', '행위의 총 편익과 해당 행위의...",3,NaN,"[희소성, 경제적 자원, 욕구, 기회 비용, 편익]"
1168,generation-for-nlp-2206,군(軍) 복무 중 원격강의를 통해 학점을 따는 사병은 연 1만명 정도로 전체 사병의...,군 복무 중 학점을 부여하는 방안에 대해 반대하는 측의 주장은 무엇인가?,"['군 복무자에게 지나친 특혜가 된다', '군 복무 중 학점 인정이 국제적으로 인정...",2,NaN,"[군 복무, 학점, 형평성, 원격강의, 특혜]"


In [ ]:
result_file_name = create_experiment_file_name(target_file=TARGET_DATA, version=EXP_VERSION)
result_file_path = os.path.join(DATA_DIR, RESULT_DIR, result_file_name)
sampled_datasets.to_csv(result_file_path, index=False)

추출된 키워드가 위키피디아에 존재하는지 여부 확인

- 키워드 추출 결과가 유효한지 확인하기 위한 작업입니다.


In [ ]:
wiki = wk.Wikipedia(user_agent=WIKI_USER_AGENT, language="ko")

In [ ]:
keywords_expanded = pd.DataFrame(sampled_datasets["keywords"].tolist(), index=sampled_datasets.index)
keywords_expanded.columns = [f"keyword_{i+1}" for i in range(keywords_expanded.shape[1])]
new_sampled_datasets = pd.concat([sampled_datasets.drop(columns=["keywords"]), keywords_expanded], axis=1)

new_sampled_datasets.head()

,id,paragraph,question,choices,answer,question_plus,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5
1190,generation-for-nlp-2237,대안학교는 중·고교만 있는 게 아니다. 20대를 위한 대안학교도 있다. 급변하는 지...,융합인재사관학교(융합사)의 설립 목적은 무엇인가?,"['청년인재 양성', '대학 진학 지원', '취업 알선', '예술 교육 강화', '...",1,NaN,융합인재사관학교,청년인재,교육,멀티플레이어,서바이벌 스토리
242,generation-for-nlp-762,대일 수출 달러에 대한 수요 달러의 가치,일본 경제가 장기적인 경기 침체를 겪고 있다고 가정하겠습니다. 일본 가계 소득 감소...,"['감소 \xa0\xa0\xa0 감소 \xa0\xa0\xa0 감소', '감소 \xa...",1,NaN,대일 수출,달러,엔화,경기 침체,가계 소득
876,generation-for-nlp-1712,추가 서비스마다 요금을 받기로 유명한 저비용항공사(LCC) 스피릿항공이 미국 소비자...,미국 소비자 만족도 조사에서 최하위를 기록한 저비용항공사는 무엇인가?,"['스피릿항공', '제트블루항공', '델타항공', '아메리칸항공', '유나이티드항공']",1,NaN,스피릿항공,제트블루항공,소비자만족도,저비용항공사,ACSI
264,generation-for-nlp-791,희소성은 다음과 같이 정의할 수 있습니다.,희소성에 대한 정의로 옳은 것은?,"['제한된 욕구와 제한된 경제적 자원 간 차이.', '행위의 총 편익과 해당 행위의...",3,NaN,희소성,경제적 자원,욕구,기회 비용,편익
1168,generation-for-nlp-2206,군(軍) 복무 중 원격강의를 통해 학점을 따는 사병은 연 1만명 정도로 전체 사병의...,군 복무 중 학점을 부여하는 방안에 대해 반대하는 측의 주장은 무엇인가?,"['군 복무자에게 지나친 특혜가 된다', '군 복무 중 학점 인정이 국제적으로 인정...",2,NaN,군 복무,학점,형평성,원격강의,특혜


In [ ]:
for col in keywords_expanded.columns:
    new_sampled_datasets[f"{col}_exists"] = new_sampled_datasets[col].apply(lambda x: wiki.page(x).exists())

new_sampled_datasets.head()

,id,paragraph,question,choices,answer,question_plus,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_1_exists,keyword_2_exists,keyword_3_exists,keyword_4_exists,keyword_5_exists
1190,generation-for-nlp-2237,대안학교는 중·고교만 있는 게 아니다. 20대를 위한 대안학교도 있다. 급변하는 지...,융합인재사관학교(융합사)의 설립 목적은 무엇인가?,"['청년인재 양성', '대학 진학 지원', '취업 알선', '예술 교육 강화', '...",1,NaN,융합인재사관학교,청년인재,교육,멀티플레이어,서바이벌 스토리,False,False,True,True,False
242,generation-for-nlp-762,대일 수출 달러에 대한 수요 달러의 가치,일본 경제가 장기적인 경기 침체를 겪고 있다고 가정하겠습니다. 일본 가계 소득 감소...,"['감소 \xa0\xa0\xa0 감소 \xa0\xa0\xa0 감소', '감소 \xa...",1,NaN,대일 수출,달러,엔화,경기 침체,가계 소득,False,True,True,True,True
876,generation-for-nlp-1712,추가 서비스마다 요금을 받기로 유명한 저비용항공사(LCC) 스피릿항공이 미국 소비자...,미국 소비자 만족도 조사에서 최하위를 기록한 저비용항공사는 무엇인가?,"['스피릿항공', '제트블루항공', '델타항공', '아메리칸항공', '유나이티드항공']",1,NaN,스피릿항공,제트블루항공,소비자만족도,저비용항공사,ACSI,True,True,False,True,False
264,generation-for-nlp-791,희소성은 다음과 같이 정의할 수 있습니다.,희소성에 대한 정의로 옳은 것은?,"['제한된 욕구와 제한된 경제적 자원 간 차이.', '행위의 총 편익과 해당 행위의...",3,NaN,희소성,경제적 자원,욕구,기회 비용,편익,True,False,True,True,True
1168,generation-for-nlp-2206,군(軍) 복무 중 원격강의를 통해 학점을 따는 사병은 연 1만명 정도로 전체 사병의...,군 복무 중 학점을 부여하는 방안에 대해 반대하는 측의 주장은 무엇인가?,"['군 복무자에게 지나친 특혜가 된다', '군 복무 중 학점 인정이 국제적으로 인정...",2,NaN,군 복무,학점,형평성,원격강의,특혜,True,True,True,False,False


In [ ]:
# _exists 열만 추출
exists_columns = [col for col in new_sampled_datasets.columns if col.endswith("_exists")]

true_count = new_sampled_datasets[exists_columns].sum().sum()  # True 값의 총합
false_count = (new_sampled_datasets[exists_columns] == False).sum().sum()  # False 값의 총합

true_count, false_count

(70, 30)